In [ ]:
from trankit import Pipeline # 1.1.0
from transformers import AutoTokenizer #4.25.1 torch 1.7.1
import pyphen #0.17.2
from collections import defaultdict
# python 3.9.12

c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
dic = pyphen.Pyphen(lang='sk_SK')

In [14]:
word='dvadsať'
len(dic.inserted(word).split("-"))

2

In [15]:
dic.inserted(word)#.split("-")

'dvad-sať'

In [16]:
p = Pipeline(lang='slovak', gpu=False)

Loading pretrained XLM-Roberta, this may take a while...
Loading tokenizer for slovak
Loading tagger for slovak
Loading lemmatizer for slovak
Active language: slovak


In [17]:
tokenizer = AutoTokenizer.from_pretrained("gerulata/slovakbert")

c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# sentences='''Ivan bodaj by si mládil slamu ako tvoj otec.'''
# p.posdep(sentences)
# id: The token’s position in the sentence (1 means it's the first word).
# text: The actual word ("Ivan").
# upos: Universal Part of Speech tag (here, "PROPN", meaning proper noun).
# xpos: Language-specific POS tag (here, "SSms1:r", possibly from a Slavic language tagging system).
# feats: Morphological features:
# Animacy=Anim → The noun is animate.
# Case=Nom → The noun is in nominative case (subject form).
# Gender=Masc → Masculine gender.
# Number=Sing → Singular form.
# head: The index of the word this token depends on (5 means it links to the 5th word in the sentence).
# deprel: Dependency relation (here, "nsubj", meaning it is the nominal subject of the sentence).
# dspan and span: Likely denote character positions in the original text (Ivan starts at 0 and ends at 4).

In [13]:
a=tokenizer.tokenize('roztopený')

In [14]:
a

['roz', 'top', 'enÃ½']

In [2]:
# sentences=''''''
# p.posdep(sentences)

In [18]:
from collections import Counter

def compute_frequency_words(text):
    words = text.lower().split() 
    word_freq = Counter(words)
    return dict(word_freq)


# print(compute_frequency_words('How are you doing ?'))

# def compute_frequency_words(content):
#     stats= len(content)
#     return stats 
# compute_frequency_words(content)

In [19]:
import re 
def preprocessing(sentences):
    sentences = re.sub(r' {2,}', ' ', sentences)
    sentences = re.sub(r'\n{2,}', '\n', sentences)
    sentences = re.sub(r'.pdf', '\n', sentences)
    sentences =re.sub(r'xa0','',sentences)
    sentences = re.sub(r'([_\-/{}}])\1+', '', sentences)
    return sentences

In [ ]:
# Source https://web.archive.org/web/20240713231039/http://arl6.library.sk/nlp4sk/help#pos_V
### XPOS ###

# ADJ: adjective
# ADP: adposition
# ADV: adverb
# AUX: auxiliary
# CCONJ: coordinating conjunction
# DET: determiner
# INTJ: interjection
# NOUN: noun
# NUM: numeral
# PART: particle
# PRON: pronoun
# PROPN: proper noun
# PUNCT: punctuation
# SCONJ: subordinating conjunction
# SYM: symbol
# VERB: verb
# X: other

### UPOS ###

# S	Podstatné meno
# A	Prídavné meno 
# P	Zámeno	
# O	Spojka	
# E	Predložka	
# V	Sloveso
	
# N	Číslovka	
# G	Príčastie	
# D	Príslovka	
# T	Častica	
# J	Citoslovce	
# R	Reflexívum	
# Y	Kondicionálna morfém
# W	Skratka	
# Z	Interpunkcia
# 0	Číslica	
    # functional=[]
    # non_functional=[]

In [ ]:
freq_content=compute_frequency_words(content)


Ordering_text

In [29]:
import json
def complexity_word(word:str):
    #tokenzation
    tokens = tokenizer.tokenize(word)
    #syllable
    if len(word) == 1:
        return 0, len(dic.inserted(word).split("-"))
    return len(word)/len(tokens), len(dic.inserted(word).split("-"))


def syntax_fun(sentence):
    description=p.posdep(sentence)
    keys_to_remove = ['id','dspan','span']
    description = {k: v for k, v in description.items() if k not in keys_to_remove}
    return description

# pád; sin/pl; osoba; čas:(pritomný čas) 
def evaluation_pos_func(xpos,):
        evaluation=0
        if len(xpos) >=4:
            if xpos[3] == 'p': # množné číslo
                evaluation+=1
        if len(xpos) >=5:
            if xpos[4] not in ['1','4']: # nominatív a akuzatív
                evaluation+=1
        ### podstatne meno ###

        ### pridavne meno ###
        elif xpos[0] == 'A': 
            if xpos[5]  == 'y': # stupeň
                evaluation+=1
            if xpos[5] == 'z': # stupeň
                evaluation+=2
            evaluation+=1
            return evaluation
        
        ### zámeno ###

        ### číslovka ###

        ### sloveso ###
        elif xpos[0] == 'V': 
            if xpos[1] in ['M','B']: # forma slovesa
                evaluation+=1
            if xpos[1] in ['H','L']:# forma slovesa
                evaluation+=2
            evaluation+=1
            return evaluation
        
        ### príčastie ###
        elif xpos[0] == 'G': 
            if xpos[5]  == 'y': # stupeň
                evaluation+=1
            if xpos[5] == 'z': # stupeň
                evaluation+=2
            evaluation+=1
            return evaluation
        
        ### prislovka ###
        elif xpos[0] in 'D': 
            if xpos[1]  == 'y': # stupeň
                evaluation+=1
            if xpos[1] == 'z': # stupeň
                evaluation+=2
            evaluation+=1
            return evaluation
        
        ### predlozka ###
        elif xpos[0] in 'E': 
            if xpos[1] == 'u': # vokalizacia
                evaluation+=1 
            evaluation += 1
            return evaluation

        evaluation+=1
        return evaluation


# def evaluation_pos_non_func(xpos):
#     return xpos

def grammar_evaluation(upos,xpos):
    if upos in ['PUNCT']:
        return 0
    if xpos in ['Y','W','X','Z']:
        return 6
    if xpos[0] in ['S','A','P','D','E','V','G','N']:
        return evaluation_pos_func(xpos)
    if xpos[0] in ['0','O','T','J','R','Y','W','Z']:
        # return evaluation_pos_non_func(xpos)
        return 1
    else:
        return 0
         


def info_from_words(description):
    ev_gram=[]
    ev_syll=[]
    ev_rt=[]
    
    head=0
    for token in description['tokens']:
        ratio_token,len_sylabes=complexity_word(token['text'])
        ev_syll.append(len_sylabes)
        ev_rt.append(ratio_token)
        ev_gram.append(grammar_evaluation(token['upos'],token['xpos']))
        if head < token['head']:
            head=token['head']

    ev_ratio_token=sum(ev_rt)/len(ev_rt)
    ev_len_sylab=sum(ev_syll)/len(ev_syll)
    ev_gram_res=sum(ev_gram)/len(ev_gram)

    return ev_gram_res,head,ev_len_sylab,ev_ratio_token

# def compute_entropy(sentence):
#     for word in 
#     return -sum((count / len(content)) * math.log2(count / len(content)) for count in freq_content.values())


def average_evaluation(dict_list):
    result = defaultdict(lambda: defaultdict(float))
    count = len(dict_list)
    for d in dict_list:
        for category, metrics in d.items():
            for key, value in metrics.items():
                result[category][key] += value

    # Compute the average
    for category, metrics in result.items():
        for key in metrics:
            result[category][key] /= count

    return dict(result)




def sentence_analysis(sentence):
    saved_sentences=[]
    description=syntax_fun(sentence)
    for s in description['sentences']:
        if len(s['text']) < 4:
            continue
        evaluation={}
        ev_gram,head,len_sylabes,ratio_token=info_from_words(s)
        evaluation['complexity']={'TTR':ratio_token}
        evaluation['syntax']={'head':head,'POS_ev':ev_gram,'syl':len_sylabes,'len_w':len(s['tokens'])}
        saved_sentences.append(evaluation)

    return saved_sentences

def computing_fre(sentences_eval):
    sum_syl = 0
    sum_len_w = 0
    for ev_s in sentences_eval:
        syntax = ev_s.get("syntax", {})  
        sum_syl += syntax.get("syl", 0)  
        sum_len_w += syntax.get("len_w", 0)

    return 206.835 - (1.015 * sum_len_w) - (84.6 * (sum_syl / sum_len_w))




        

In [ ]:
path=''
with open(path, "r", encoding="utf8") as file:
    data = json.load(file)
    for sentences in data:
        sentences=preprocessing(sentences['page'])
        if len(sentences)<2:
            continue
        sentences_eval=sentence_analysis(sentences)
        fre=computing_fre(sentences_eval)
        part_eval=average_evaluation(sentences_eval)


Masking

In [25]:
import random
def evaluation_pos_func_mask(xpos):
        evaluation=0
        if len(xpos) >=4:
            if xpos[3] == 'p': # množné číslo
                evaluation+=1
        if len(xpos) >=5:
            if xpos[4] not in ['1','4']: # nominatív a akuzatív
                evaluation+=1
        ### podstatne meno ###

        ### pridavne meno ###
        elif xpos[0] == 'A': 
            if xpos[5]  == 'y': # stupeň
                evaluation+=1
            if xpos[5] == 'z': # stupeň
                evaluation+=2
            evaluation+=1
            return evaluation
        
        ### zámeno ###

        ### číslovka ###

        ### sloveso ###
        elif xpos[0] == 'V': 
            if xpos[1] in ['M','B']: # forma slovesa
                evaluation+=1
            if xpos[1] in ['H','L']:# forma slovesa
                evaluation+=2
            evaluation+=1
            return evaluation
        
        ### príčastie ###
        elif xpos[0] == 'G': 
            if xpos[5]  == 'y': # stupeň
                evaluation+=1
            if xpos[5] == 'z': # stupeň
                evaluation+=2
            evaluation+=1
            return evaluation
        
        ### prislovka ###
        elif xpos[0] in 'D': 
            if xpos[1]  == 'y': # stupeň
                evaluation+=1
            if xpos[1] == 'z': # stupeň
                evaluation+=2
            evaluation+=1
            return evaluation
        
        ### predlozka ###
        elif xpos[0] in 'E': 
            if xpos[1] == 'u': # vokalizacia
                evaluation+=1 
            evaluation += 1
            return evaluation

        evaluation+=1
        return evaluation

def grammar_evaluation_mask(upos,xpos):
    if upos in ['PUNCT']:
        return 0
    if xpos in ['Y','W','X','Z']:
        return 6
    if xpos[0] in ['S','A','P','D','E','V','G','N']:
        return evaluation_pos_func_mask(xpos)
    if xpos[0] in ['0','O','T','J','R','Y','W','Z']:
        # return evaluation_pos_non_func(xpos)
        return 1
    else:
        return 0
    
def syntax_fun_mask(sentence):
    description=p.posdep(sentence)
    keys_to_remove = ['id','dspan','span']
    description = {k: v for k, v in description.items() if k not in keys_to_remove}
    return description
    
def select_mask(evaluation_list):
    max_value = max(evaluation_list)
    indices = [i for i, val in enumerate(evaluation_list) if val == max_value]
    mask_id=random.choice(indices)
    return mask_id



def evaluation_pos_mask(text):
    evaluation_list=[]
    description=syntax_fun_mask(text)
    for s in description['sentences']:
        for token in s['tokens']:
            evaluation=grammar_evaluation_mask(token['upos'],token['xpos'])
            evaluation_list.append(evaluation)
    mask_id=select_mask(evaluation_list)
    return mask_id


    
text="Rád sa hram a je to zábava pretože, ked budem dospelý už sa hrať nebudem."
masked_text=''
words = text.split()
for index,number in enumerate(range(0, len(words), 7)):
    chunk = words[number:number+7] 
    if len(chunk) == 7: 
        mask_id= evaluation_pos_mask(" ".join(chunk))
        chunk[mask_id]='[MASK]'
    masked_text+=f' {" ".join(chunk)}'